<img align="center" src="images/course.png" width="800">

# 16720 (B)  3D Reconstruction - Assignment 5 - P1
    Instructor: Kris                          TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

In [1]:
# Helper functions for this assignment. DO NOT MODIFY!!!
"""
Helper functions.
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import scipy.optimize



def _epipoles(E):
    U, S, V = np.linalg.svd(E)
    e1 = V[-1, :]
    U, S, V = np.linalg.svd(E.T)
    e2 = V[-1, :]
    return e1, e2

def displayEpipolarF(I1, I2, F):
    matplotlib.use('TkAgg')
    e1, e2 = _epipoles(F)

    sy, sx, _ = I2.shape

    f, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 9))
    ax1.imshow(I1)
    ax1.set_title('Select a point in this image')
    ax1.set_axis_off()
    ax2.imshow(I2)
    ax2.set_title('Verify that the corresponding point \n is on the epipolar line in this image')
    ax2.set_axis_off()

    while True:
        plt.sca(ax1)
        x, y = plt.ginput(1, timeout=3600, mouse_stop=2)[0]

        xc = x
        yc = y
        v = np.array([xc, yc, 1])
        l = F.dot(v)
        s = np.sqrt(l[0]**2+l[1]**2)

        if s==0:
            error('Zero line vector in displayEpipolar')

        l = l/s

        if l[0] != 0:
            ye = sy-1
            ys = 0
            xe = -(l[1] * ye + l[2])/l[0]
            xs = -(l[1] * ys + l[2])/l[0]
        else:
            xe = sx-1
            xs = 0
            ye = -(l[0] * xe + l[2])/l[1]
            ys = -(l[0] * xs + l[2])/l[1]

        # plt.plot(x,y, '*', 'MarkerSize', 6, 'LineWidth', 2);
        ax1.plot(x, y, '*', markersize=6, linewidth=2)
        ax2.plot([xs, xe], [ys, ye], linewidth=2)
        plt.draw()


def _singularize(F):
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    F = U.dot(np.diag(S).dot(V))
    return F

def _objective_F(f, pts1, pts2):
    F = _singularize(f.reshape([3, 3]))
    num_points = pts1.shape[0]
    hpts1 = np.concatenate([pts1, np.ones([num_points, 1])], axis=1)
    hpts2 = np.concatenate([pts2, np.ones([num_points, 1])], axis=1)
    Fp1 = F.dot(hpts1.T)
    FTp2 = F.T.dot(hpts2.T)

    r = 0
    for fp1, fp2, hp2 in zip(Fp1.T, FTp2.T, hpts2):
        r += (hp2.dot(fp1))**2 * (1/(fp1[0]**2 + fp1[1]**2) + 1/(fp2[0]**2 + fp2[1]**2))
    return r

def refineF(F, pts1, pts2):
    f = scipy.optimize.fmin_powell(
        lambda x: _objective_F(x, pts1, pts2), F.reshape([-1]),
        maxiter=100000,
        maxfun=10000,
        disp = False
    )
    return _singularize(f.reshape([3, 3]))

def camera2(E):
    U,S,V = np.linalg.svd(E)
    m = S[:2].mean()
    E = U.dot(np.array([[m,0,0], [0,m,0], [0,0,0]])).dot(V)
    U,S,V = np.linalg.svd(E)
    W = np.array([[0,-1,0], [1,0,0], [0,0,1]])

    if np.linalg.det(U.dot(W).dot(V))<0:
        W = -W

    M2s = np.zeros([3,4,4])
    M2s[:,:,0] = np.concatenate([U.dot(W).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,1] = np.concatenate([U.dot(W).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,2] = np.concatenate([U.dot(W.T).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,3] = np.concatenate([U.dot(W.T).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    return M2s


def calc_epi_error(pts1_homo, pts2_homo, F):
    '''
    Helper function to calculate the sum of squared distance between the corresponding points and the estimated epipolar lines. 
    Expect pts1 and pts2 are in homogeneous coordinates and not normalized. 
    '''
    line1s = pts1_homo.dot(F.T)
    dist1 = np.square(np.divide(np.sum(np.multiply(
        line1s, pts2_homo), axis=1), np.linalg.norm(line1s[:, :2], axis=1)))

    line2s = pts2_homo.dot(F)
    dist2 = np.square(np.divide(np.sum(np.multiply(
        line2s, pts1_homo), axis=1), np.linalg.norm(line2s[:, :2], axis=1)))

    ress = (dist1 + dist2).flatten()
    return ress


def toHomogenous(pts):
    return np.vstack([pts[:,0],pts[:,1],np.ones(pts.shape[0])]).T.copy()

# Coding P1: Fundamental Matrix Estimation

## Overview
In this part, you will implement two different methods seen in class to estimate the fundamental matrix from the corresponding points in a pair images. In the data/ directory, you will find two images: 

|![alt](images/im1.png) |![alt](images/im2.png)|
|-|-|

from the [Middlebury multiview dataset](http://vision.middlebury.edu/mview/data/)., which is used to evaluate the performance of modern 3D reconstruction algorithms.



## Q1: Fundamental matrix estimation
### Q1.1: The Eight Point Algorithm (2 pt writeup, 8 pt implementation)
The 8-point algorithm (discussed in class, and outlined in Section 10.1 of Forsyth & Ponce) is arguably the simplest method for estimating the fundamental matrix. For this section, you can use provided correspondences you can find in data/some corresp.npz. Write the function: 

```
            F = eightpoint(pts1, pts2, m)
```
where pts1 and pts2 are $N \times 2$ matrices corresponding to the $(x,y)$ coordinates of the $N$ points in the first and second image respectively. **M** is a scale parameter.

-  You should scale the data as was discussed in class, by dividing each
    coordinate by $m$ (the maximum of the image's width and height). After
    computing $\textbf{F}$, you will have to "unscale" the fundamental matrix.
 **Hint:** If $\textbf{x}_{normalized} = \textbf{T}\textbf{x}$, then $\textbf{F}_{unnormalized} = \textbf{T}^T \textbf{F} \textbf{T}$. $\textbf{T}$ is a $3 \times 3$ diagonal matrix formed from $\texttt{m}$. 
     
     You must enforce the singularity condition of the $\textbf{F}$ before unscaling.
     
- You may find it helpful to refine the solution by using local
    minimization.  This probably won't fix a completely broken solution, but may make a good solution better by locally minimizing a geometric cost function. For this, we have provided a helper function `refineF` in taking in $\textbf{F}$ and two sets of points, which you can call from `eightpoint` before unscaling `F`.
  
- Remember that the $x$-coordinate of a point in the image is its column entry, and $y$-coordinate is the row entry. Also note that eight-point is just a figurative name, it just means that you need at least 8 points; your algorithm should use an over-determined system ($N>8$ points).
    
- To visualize the correctness of your estimated $\textbf{F}$, use the supplied function `displayEpipolarF`, which takes in F, and the two images. This GUI lets you select a point in one of the images and visualize the corresponding epipolar line in the other image.

- In addition to visualization, we also provide a helper function `calc_epi_error` to evaluate the quality of the estimated fundamental matrix. This function calculates the distance between the estimated epipolar line and the corresponding points. For the eight point algorithm, the error should on average be < 1. 

<img align="center" src="images/epi.png" width="800">

**Important:** <span style='color:red'>Notice that since we are using Jupyter, it is a bit tricky to have a functional GUI.</span> Here we use the `matplotlib.use('TkAgg')` as backend for launching an external gui plot. You can close the popup gui to end the gui process. In the case where the gui hangs and does not close naturally, restart the notebook kernel usually does the trick. 

Reference to install tkinker in your system if you are having issues [link](https://stackoverflow.com/questions/4783810/install-tkinter-for-python). 

<span style='color:red'>**Output:**</span> In your write-up: Write your recovered $\textbf{F}$ and include an image of some example outputs of displayEpipolarF.

In [2]:
def eightpoint(pts1, pts2, M):
    '''
    Q1.1: Eight Point Algorithm for calculating the fundamental matrix
        Input:  pts1, Nx2 Matrix containing the corresponding points from image1
                pts2, Nx2 Matrix containing the corresponding points from image2
                M, a scalar parameter computed as max (imwidth, imheight)
        Output: F, the fundamental matrix of shape (3, 3)
    
    ***
    HINTS:
    (1) Normalize the input pts1 and pts2 using the matrix T.
    (2) Setup the eight point algorithm's equation.
    (3) Solve for the least square solution using SVD. 
    (4) Use the function `_singularize` (provided) to enforce the singularity condition. 
    (5) Use the function `refineF` (provided) to refine the computed fundamental matrix. 
        (Remember to usethe normalized points instead of the original points)
    (6) Unscale the fundamental matrix

    '''

    F = None # output fundamental matrix 
    N = pts1.shape[0] # Extrating the number of points  
    T = np.diag([1/M, 1/M, 1])
    
    # Converting the points to homogenous coordinates
    pts1_homogenous, pts2_homogenous = toHomogenous(pts1), toHomogenous(pts2)
    pts1_homogenous = np.dot(T, pts1_homogenous.T).T
    pts2_homogenous = np.dot(T, pts2_homogenous.T).T
    # Computing the 3x3 matrix used to normalize corresponding points. 
    
    A = []
    for i in range(len(pts1_homogenous)):
        temp = []
        pt1 = pts1_homogenous[i]
        pt2 = pts2_homogenous[i]

        temp.append(pt1[0] * pt2[0])
        temp.append(pt1[0] * pt2[1])
        temp.append(pt1[0] )
        temp.append(pt1[1] * pt2[0])
        temp.append(pt1[1] * pt2[1])
        temp.append(pt1[1] )
        temp.append(pt2[0])
        temp.append(pt2[1])
        temp.append(1)
        A.append(temp)

    A = np.array(A)

    U, S, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3).T
    
    F = _singularize(F)  
    
    F = refineF(F, pts1/M, pts2/M)
    F = np.dot(np.dot(T.T, F), T)
    
    F = F/F[2,2] #Finding the unique fundamental matrix by setting the scale to 1. 
    return F




In [3]:
# Load images and visualize epipolar lines. 
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']

im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')
F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
print(F)
N = pts1.shape[0]
pts1_homogenous, pts2_homogenous = toHomogenous(pts1), toHomogenous(pts2)
print("Error:", np.mean(calc_epi_error(pts1_homogenous, pts2_homogenous, F)))

## Important!! Uncomment this line to visualize, but before you submit, 


[[-0.      0.     -0.2519]
 [ 0.     -0.      0.0026]
 [ 0.2422 -0.0068  1.    ]]
Error: 0.39895034989884903


In [4]:
# displayEpipolarF(im1, im2, F)

KeyboardInterrupt: 

In [6]:
# Simple Tests to verify your implmentation:

correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
print(F)
pts1_homogenous, pts2_homogenous = toHomogenous(pts1), toHomogenous(pts2)
print(np.mean(calc_epi_error(pts1_homogenous, pts2_homogenous, F)))
assert(F.shape == (3, 3))
assert(F[2, 2] == 1)
assert(np.linalg.matrix_rank(F) == 2)
assert(np.mean(calc_epi_error(pts1_homogenous, pts2_homogenous, F)) < 1)

[[-0.      0.     -0.2519]
 [ 0.     -0.      0.0026]
 [ 0.2422 -0.0068  1.    ]]
0.39895034989884903


In [ ]:
# Hidden Tests

### Q1.2: The Seven Point Algorithm (5 pt writeup, 10 pt implementation)

Since the fundamental matrix only has seven degrees of freedom, it is possible to calculate $\textbf{F}$ using only seven point correspondences. This requires solving a polynomial equation.  In the section, you will implement the seven-point algorithm (outlined in this [post](https://imkaywu.github.io/blog/2017/06/fundamental-matrix/)). Manually select $7$ points from provided point in `data/some_corresp.npz`, and use these points to recover a fundamental matrix $\textbf{F}$. The function should have the signature:

```
Farray = sevenpoint(pts1, pts2, m)
```

where pts1 and pts2 are $7 \times 2$ matrices containing the correspondences and $m$ is the normalizer (use the maximum of the images' height and width), and `Farray` is a list array of length either 1 or 3 containing Fundamental matrix/matrices. Use $m$ to normalize the point values between $[0,1]$ and remember to "unnormalize" your computed $\textbf{F}$ afterwards.

- Use `calc_epi_error` to calculate the error pick the best one, and use `displayEpipolarF` to visualize and verify the solution.

<span style='color:red'>**Output:**</span> In your write-up: Print your recovered $\textbf{F}$ and include an image output of `displayEpipolarF`.


In [7]:
def sevenpoint(pts1, pts2, M):
    '''
    Q1.2: Seven Point Algorithm for calculating the fundamental matrix
        Input:  pts1, 7x2 Matrix containing the corresponding points from image1
                pts2, 7x2 Matrix containing the corresponding points from image2
                M, a scalar parameter computed as max (imwidth, imheight)
        Output: Farray, a list of estimated 3x3 fundamental matrixes.
        
    ***
    HINTS:
    (1) Normalize the input pts1 and pts2 scale paramter M.
    (2) Setup the seven point algorithm's equation.
    (3) Solve for the least square solution using SVD. 
    (4) Pick the last two colum vector of vT.T (the two null space solution f1 and f2)
    (5) Use the singularity constraint to solve for the cubic polynomial equation of  F = a*f1 + (1-a)*f2 that leads to 
        det(F) = 0. Sovling this polynomial will give you one or three real solutions of the fundamental matrix. 
        Use np.polynomial.polynomial.polyroots to solve for the roots
    (6) Unscale the fundamental matrixes and return as Farray
    '''

    Farray = []

    T = np.diag([1/M, 1/M, 1])
    
    # Converting the points to homogenous coordinates
    pts1_homogenous, pts2_homogenous = toHomogenous(pts1), toHomogenous(pts2)
    pts1_homogenous = np.dot(T, pts1_homogenous.T).T
    pts2_homogenous = np.dot(T, pts2_homogenous.T).T
    A = []
    for i in range(len(pts1_homogenous)):
        temp = []
        pt1 = pts1_homogenous[i]
        pt2 = pts2_homogenous[i]

        temp.append(pt1[0] * pt2[0])
        temp.append(pt1[0] * pt2[1])
        temp.append(pt1[0] )
        temp.append(pt1[1] * pt2[0])
        temp.append(pt1[1] * pt2[1])
        temp.append(pt1[1] )
        temp.append(pt2[0])
        temp.append(pt2[1])
        temp.append(1)
        A.append(temp)

    A = np.array(A)

    U, S, V = np.linalg.svd(A)
    f2 = V[-1].reshape(3, 3).T
    f1 = V[-2].reshape(3, 3).T
    
    # F = a*f1 + (1-a)*f2
    lam = lambda a : np.linalg.det(a*f1 + (1 - a)*f2)


    a0 = lam(0)
    a1 = 2*(lam(1) - lam(-1))/3 - (lam(2) - lam(-2))/12
    a2 = (lam(1) + lam(-1)) / 2 - a0
    a3 =  (lam(1) - lam(-1)) / 2 - a1


    

    roots = np.roots((a3, a2, a1, a0))
    Farray = []

    for a in roots:
        if np.isreal(a):
            a = np.real(a)
            F = a*f1 + (1 - a)*f2
                

            F = refineF(F, pts1/M, pts2/M)

            F = np.dot(np.dot(T.T, F), T)
            
            F /= F[2, 2]

            Farray.append(F)


    return Farray


In [10]:
# Full set of tests; you will get full points for coding if you pass the following tests. 

# Test out the seven-point algorithm by randomly sampling 7 points and finding the best solution. 
np.random.seed(1)
np.set_printoptions(precision=4, suppress=1)

correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

max_iter = 500
pts1_homo = np.hstack((pts1, np.ones((pts1.shape[0], 1))))
pts2_homo = np.hstack((pts2, np.ones((pts2.shape[0], 1))))

ress = []
F_res = []
choices = []
minres = 5000
F_out = None
M=np.max([*im1.shape, *im2.shape])
for i in range(max_iter):
    choice = np.random.choice(range(pts1.shape[0]), 7)
    pts1_choice = pts1[choice, :]
    pts2_choice = pts2[choice, :]
    Fs = sevenpoint(pts1_choice, pts2_choice, M)
    for F in Fs:
        choices.append(choice)
        res = np.mean(calc_epi_error(pts1_homo,pts2_homo, F))
        if res < minres:
            F_out = F
            minres = res
        if(res < 1):
            break
        
F = F_out
print("Error = ", minres)
assert(F.shape == (3, 3))
assert(F[2, 2] == 1)
assert(np.linalg.matrix_rank(F) == 2)
assert(np.mean(calc_epi_error(pts1_homogenous, pts2_homogenous, F)) < 1)


Error =  0.5668901239522244


In [12]:
print(F)
displayEpipolarF(im1, im2, F)

[[ 0.      0.     -0.201 ]
 [ 0.     -0.      0.0007]
 [ 0.1922 -0.0042  1.    ]]


IndexError: list index out of range